In [1]:
import os
import requests
import pandas as pd
import numpy as np 
import boto3

bsdir = "s3://ipsos-dvd/"
data_dir = bsdir + "ppl/data/"
data_ppl = "s3://external-people-data-labs/"


from pyspark import SparkContext
from pyspark.ml.feature import StringIndexer


from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType,
    LongType,
    StructField,
    IntegerType,
    StringType,
    DoubleType,
    TimestampType,
    ArrayType,
    MapType
)
import pyspark.sql.functions as F


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1706716884988_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Play with company data

In [6]:
# try with this option
# .option("quote", "\"")
# .option("escape", "\"")

comp = spark.read.option("quote", "\"").option("escape", "\"").csv(data_ppl + "company-data-and-insights", header=True) # this is necessary because the fields contain "," as well but dont have quotes around them


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# comp = (sqlContext.read.format('com.databricks.spark.csv')
#          .options(header='true', inferschema='true')
#          .option('escape','"')
#         .load(data_ppl + "company-data-and-insights//*.gz")) # this is necessary because the fields contain "," as well but dont have quotes around them


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# additions-departures over time

# json_schema = MapType(StringType(), IntegerType())
# json_schema = spark.read.json(comp.rdd.map(lambda row: row.GROSS_ADDITIONS_BY_MONTH)).schema

# Define a UDF to parse JSON and extract values
def parse_json_to_array(json_str):
    try:
        json_obj = json.loads(json_str)
        return list(json_obj.values())
    except:
        return None
    
parse_udf = F.udf(parse_json_to_array, ArrayType(IntegerType()))



comp = comp.withColumn("GROSS_ADDITIONS_BY_MONTH", parse_udf(F.col("GROSS_ADDITIONS_BY_MONTH")))
# comp = comp.withColumn("GROSS_DEPARTURES_BY_MONTH",  F.map_values(F.from_json(comp["GROSS_DEPARTURES_BY_MONTH"], json_schema)))

# churn = comp.select("GROSS_ADDITIONS_BY_MONTH", "GROSS_DEPARTURES_BY_MONTH", "ID")

# churn = (churn.withColumn("tmp", F.arrays_zip("GROSS_ADDITIONS_BY_MONTH", "GROSS_DEPARTURES_BY_MONTH"))
#     .withColumn("tmp", )
#     .withColumn("tmp", F.explode("tmp"))
#     .select("ID", F.col("tmp.GROSS_DEPARTURES_BY_MONTH"), F.col("tmp.GROSS_ADDITIONS_BY_MONTH")))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
comp.select("GROSS_ADDITIONS_BY_MONTH").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|GROSS_ADDITIONS_BY_MONTH|
+------------------------+
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
|                    null|
+------------------------+
only showing top 20 rows

In [ ]:

comp = comp.withColumn("GROSS_ADDITIONS_BY_MONTH", F.from_json("GROSS_ADDITIONS_BY_MONTH"))
comp = comp.withColumn("GROSS_DEPARTURES_BY_MONTH", F.from_json("GROSS_DEPARTURES_BY_MONTH"))

In [ ]:
comp.select("GROSS_ADDITIONS_BY_MONTH").show(200)

In [31]:
temp = comp.limit(10).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
temp.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                     NAME  ...                         TOP_NEXT_EMPLOYERS_BY_ROLE
0                         pga outsourcing  ...  "{\"all\":{\"bennett-thrasher\":1,\"ernstandyo...
1          swimming pools by jack anthony  ...  "{\"all\":{\"cyclomedia\":1,\"holland-bloorvie...
2                                 dynalco  ...  "{\"all\":{\"compressor-engineering-corporatio...
3                              kubo cloud  ...                                               None
4  we believe in the opensource community  ...                                               None

[5 rows x 46 columns]

In [21]:
temp['GROSS_DEPARTURES_BY_MONTH']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0    "{\"2011-11\":1,\"2011-12\":0,\"2012-01\":0,\"...
1    "{\"2013-07\":1,\"2013-08\":0,\"2013-09\":0,\"...
2    "{\"2010-03\":1,\"2010-04\":0,\"2010-05\":0,\"...
3                                                 None
4                                                 None
5                                    "{\"2023-07\":0}"
6                                    "{\"2023-07\":0}"
7    "{\"2010-09\":1,\"2010-10\":0,\"2010-11\":0,\"...
8    "{\"2012-07\":2,\"2012-08\":0,\"2012-09\":0,\"...
9                                                 None
Name: GROSS_DEPARTURES_BY_MONTH, dtype: object